In [3]:
#import xgboost as xgb
#from sklearn import datasets
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder 
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.utils import class_weight


data_path = 'C:/Users/mpar0031/Work/Data/Tabular/'
fn = 'arrhythmia.data'
#var_names = 'arrhythmia.names'
df = pd.read_csv(data_path + fn, header=None)

# last column is the class label
X = df.iloc[:,:-1]

numeric_features = []
categorical_features = []
for col in X.columns:
    dt = X[col].dtype
    if dt == 'object':
        X[col] = X[col].fillna('NA')
        categorical_features.append(col)
    else:
        numeric_features.append(col)

cat_coder = OneHotEncoder()
X_cat = X.loc[:, categorical_features]
X_cat.columns = 'CAT_' + X_cat.columns.astype(str)
categorical_features = list(X_cat.columns)
X_cat = cat_coder.fit_transform(X_cat)
cat_feat_names = cat_coder.get_feature_names_out(categorical_features)

X = df.loc[:, numeric_features]  
X = pd.concat([X, pd.DataFrame(X_cat.todense(), columns=cat_feat_names)], axis=1)
y = df.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=11)
lc = LabelEncoder() 
lc = lc.fit(y) 
lc_y = lc.transform(y)

classes_weights = class_weight.compute_sample_weight(class_weight='balanced', y=y_train)
model = XGBClassifier(objective='multi:softmax', n_estimators=100, 
                      max_depth=3, subsample=0.9, learning_rate=0.1, 
                      reg_alpha=0.001, reg_lambda=0.001, gamma=2, 
                      min_child_weight=8)

model.fit(X_train, y_train, sample_weight=classes_weights)
y_pred = model.predict(X_test) 
predictions = [round(value) for value in y_pred]
accuracy = balanced_accuracy_score(y_test, predictions) 

print("Accuracy: %.2f%%" % (accuracy * 100.0))

C:\Users\mpar0031\Work\Softwares\WPy64-3830\python-3.8.3.amd64\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:51:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 63.32%


C:\Users\mpar0031\Work\Softwares\WPy64-3830\python-3.8.3.amd64\lib\site-packages\sklearn\metrics\_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
